## Various ways to get Options expiration dates

In [1]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import BDay
import pandas_datareader.data as pdr
import datetime
import pytz
import pathlib
from dateutil.relativedelta import *
import pandas_market_calendars as pmc
import io

In [2]:
uk_holidays = open('expiration_data/uk_holidays.csv').readlines()
# uksplit = [','.join([t.strip('\n') for t in l.split(',')]) for l in  uk_holidays]
uksplit = [','.join(l.split(',')) for l in  uk_holidays]
fio = io.StringIO()
fio.writelines(uksplit)
fio.seek(0)
df_ukh = pd.read_csv(fio)
def ukh_to_yyyymmdd(month_day,year):
    md = month_day.strip()
    d = datetime.datetime.strptime(f'{md} {year}', '%B %d %Y')
    return d
#     return d.strftime("%Y-%m-%d")

year_cols = [c for c in df_ukh.columns.values if '20' in str(c)]
for c in year_cols:
    df_ukh[c] = df_ukh[c].apply(lambda s:ukh_to_yyyymmdd(s,c))
uk_holidays = sorted(df_ukh[year_cols].values.reshape(-1))
uk_holidays

[numpy.datetime64('2020-01-02T00:00:00.000000000'),
 numpy.datetime64('2020-04-10T00:00:00.000000000'),
 numpy.datetime64('2020-04-13T00:00:00.000000000'),
 numpy.datetime64('2020-05-04T00:00:00.000000000'),
 numpy.datetime64('2020-05-25T00:00:00.000000000'),
 numpy.datetime64('2020-08-31T00:00:00.000000000'),
 numpy.datetime64('2020-11-30T00:00:00.000000000'),
 numpy.datetime64('2020-12-25T00:00:00.000000000'),
 numpy.datetime64('2020-12-28T00:00:00.000000000'),
 numpy.datetime64('2021-01-01T00:00:00.000000000'),
 numpy.datetime64('2021-04-02T00:00:00.000000000'),
 numpy.datetime64('2021-04-05T00:00:00.000000000'),
 numpy.datetime64('2021-05-03T00:00:00.000000000'),
 numpy.datetime64('2021-05-31T00:00:00.000000000'),
 numpy.datetime64('2021-08-30T00:00:00.000000000'),
 numpy.datetime64('2021-11-30T00:00:00.000000000'),
 numpy.datetime64('2021-12-27T00:00:00.000000000'),
 numpy.datetime64('2021-12-28T00:00:00.000000000'),
 numpy.datetime64('2022-01-03T00:00:00.000000000'),
 numpy.datet

In [3]:
MONTH_CODES = 'FGHJKMNQUVXZ'
DICT_MONTH_CODE = {MONTH_CODES[i]:i+1 for i in range(len(MONTH_CODES))}

from pandas.tseries.holiday import USFederalHolidayCalendar
bday_us = pd.offsets.CustomBusinessDay(calendar=USFederalHolidayCalendar())
# bday_uk = pd.offsets.CustomBusinessDay(calendar=pmc.exchange_calendar_ice.ICEExchangeCalendar().regular_holidays)
bday_uk = pd.offsets.CustomBusinessDay(holidays=uk_holidays)
TIMEZONE = 'US/Eastern'


In [1]:

def get_nth_weekday(year,month,target_weekday,nth_occurrence):
    '''
    weekday is the term that assigns numbers from 0 to 6 to the days of the weeks.
    weekday 0 = monday
    '''
    # get dayofweeks of year,month,1
    weekday_01 = datetime.datetime(year,month,1).weekday()
    if weekday_01 <= target_weekday:
        day_of_month_of_first_occurence = target_weekday - weekday_01
        day_of_month_of_nth_occurence = day_of_month_of_first_occurence + 1 + (nth_occurrence - 1) * 7
    else:
        day_of_month_of_nth_occurence = target_weekday - weekday_01 + 1 + (nth_occurrence) * 7 
    return datetime.datetime(year,month,day_of_month_of_nth_occurence)




def get_ES_expiry(symbol):
    '''
    3rd friday of month of symbol
    '''
    monthcode_yy = symbol[2:]
    month = DICT_MONTH_CODE[monthcode_yy[0]]
    year = 2000 + int(monthcode_yy[1:])
    return get_nth_weekday(year,month,4,3)

def get_E6_expiry(symbol):
    monthcode_yy = symbol[2:]
    next_month = DICT_MONTH_CODE[monthcode_yy[0]] + 1
    year = 2000 + int(monthcode_yy[1:])
    if next_month>12:
        next_month = 1
        year += 1
    return datetime.datetime(year,next_month,1) - 7*bday_us

def get_CL_expiry(symbol):
    '''
    Trading terminates 7 business days before the 26th calendar of the month prior to the contract month.
    '''
    monthcode_yy = symbol[2:]
    month = DICT_MONTH_CODE[monthcode_yy[0]]
    year = 2000 + int(monthcode_yy[1:])
    month = month -1
    if month<1:
        month = 12
        year = year - 1
    return datetime.datetime(year,month,26) - 7*bday_us

def get_NG_expiry(symbol):
    monthcode_yy = symbol[2:]
    month = DICT_MONTH_CODE[monthcode_yy[0]]
    year = 2000 + int(monthcode_yy[1:])
    return datetime.datetime(year,month,1) - 4*bday_us

def get_CB_expiry(symbol):
    '''
    This is the spec for the CME Brent, but it matches ICE.
    Trading terminates the 4th last London business day of 
    the month, 2 months prior to the contract month 
    except for the February contract month which 
    terminates the 5th last London business day of the 
    month, 2 months prior to the contract month.  
    '''
    monthcode_yy = symbol[2:]
    month = DICT_MONTH_CODE[monthcode_yy[0]]
    year = 2000 + int(monthcode_yy[1:])
    month = month - 1
    if month<1:
        month = 12 + month
        year = year - 1
    days_to_subtract = 4
    if monthcode_yy[0] =='G':
        days_to_subtract = 5
    elif monthcode_yy[0] == 'F':
        days_to_subtract = 3
#     elif monthcode_yy == 'N22':
#         days_to_subtract = 7
    return datetime.datetime(year,month,1,0,0) - days_to_subtract * bday_uk

DICT_PRODUCT = {
    'E6':get_E6_expiry,
    'ES':get_ES_expiry,
    'CL':get_CL_expiry,
    'NG':get_NG_expiry,
    'CB':get_CB_expiry,
}

    
def get_expiry(symbol):
    product = symbol[:2]
    f = DICT_PRODUCT[product]
    return f(symbol)


def dt_from_yyyymmdd(yyyymmdd,hour=0,minute=0,timezone=TIMEZONE):
    y = int(str(yyyymmdd)[0:4])
    m = int(str(yyyymmdd)[4:6])
    d = int(str(yyyymmdd)[6:8])  
    return datetime.datetime(y,m,d,hour,minute,tzinfo=pytz.timezone(timezone))

def yyyymmdd_from_dt(dt):
    y = int(dt.year)
    m = int(dt.month)
    d = int(dt.day)
    return y*100*100 + m*100 + d

def get_dte_pct(trade_yyyymmdd,expiry_yyyymmdd):
    dt_td = dt_from_yyyymmdd(trade_yyyymmdd)
    dt_xp = dt_from_yyyymmdd(expiry_yyyymmdd)
    return ((dt_xp - dt_td).days + 1)/365


NameError: name 'TIMEZONE' is not defined

In [5]:
# get options expirations from barchart using curl
# c = 'CB'
# ybeg = 20
# yend = 23
# for y in range(ybeg,yend+1):
#     !bash barchart_expirations.sh {c} {y}

In [6]:
c = "CB"
ybeg = 20
yend = 23
df_exp = None
for y in range(ybeg,yend+1):
    dft = pd.read_csv(f'expiration_data/commod_expirations_{c}{y}.csv')
    if df_exp is None:
        df_exp = dft.copy()
    else:
        df_exp = df_exp.append(dft,)
def mmddyy_to_yyyymmdd(mmddyy):
    if str(mmddyy).lower() == 'nan':
        return 99999999
    s = mmddyy.split('/')
    y =  2000 + int(s[2])
    m = int(s[0])
    d = int(s[1])
    return (y*100*100 + m*100 + d)
def yyyymmdd_from_symbol(s):
    d = get_expiry(s)
    return int(d.year*100*100 + d.month*100 + d.day)
df_exp.expiry = df_exp.expiry.apply(lambda mmddyy:mmddyy_to_yyyymmdd(mmddyy))
df_exp['theo_expiry'] = df_exp.symbol.apply(lambda s:yyyymmdd_from_symbol(s))
df_exp['theo_is_good'] = df_exp.apply(lambda r: False if (r.expiry==99999999) or (r.expiry != r.theo_expiry) else True,axis=1)
df_exp = df_exp[df_exp.expiry!=99999999]
df_exp

,symbol,expiry,theo_expiry,theo_is_good
4,CBK20,20200326,20200326,True
5,CBM20,20200427,20200427,True
6,CBN20,20200526,20200526,True
7,CBQ20,20200625,20200625,True
8,CBU20,20200728,20200728,True
9,CBV20,20200825,20200825,True
10,CBX20,20200925,20200925,True
11,CBZ20,20201027,20201027,True
0,CBF21,20201125,20201125,True
1,CBG21,20201223,20201223,True
